# Tovana Memory 🧠 with LangGraph 🦜🕸️: Cross-Thread Memory Sharing

Welcome to this cookbook on integrating Tovana Memory with LangGraph! In this tutorial, we'll explore how to use Tovana's memory and belief system within a LangGraph agent, demonstrating cross-thread memory sharing using LangGraph's wider channel memory feature.

This cookbook is based on the [LangGraph Wider Channels Memory Cookbook](https://github.com/langchain-ai/langgraph/blob/2a46c60551dd6baca9cbb02a7bb6effebb033d62/examples/memory/wider-channels.ipynb?short_path=8f86cb3), adapted to showcase Tovana's unique features.

## What we'll cover:

1. Installing required packages
2. Setting up the environment
3. Initializing Tovana MemoryManager
4. Creating a LangGraph agent with Tovana memory
5. Demonstrating cross-thread memory sharing
6. Analyzing the results

A key feature we'll demonstrate is batch message saving. Instead of saving every message individually, we'll save messages in batches. This approach preserves context, saves processing time and costs, and often leads to better results as it allows the system to understand the full context of a conversation before updating the memory.

Let's get started!

## 1. Installing required packages

First, let's install the necessary packages. Run the following cell to install LangChain, LangGraph, Tovana, and LangChain OpenAI:

In [ ]:
!pip install langchain langgraph tovana langchain-openai

print("Required packages installed.")

## 2. Setting up the environment

Now that we have the required packages installed, let's import the necessary libraries and set up our environment:

In [ ]:
import os
import uuid
from typing import Annotated, Any, Dict

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.graph import END, START
from langgraph.graph.message import MessagesState
from langgraph.graph.state import StateGraph
from langgraph.managed.shared_value import SharedValue
from langgraph.store.memory import MemoryStore

from memory import MemoryManager

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

print("Environment setup complete.")

## 3. Initializing Tovana MemoryManager

Now, let's initialize the Tovana MemoryManager with our specific configuration:

In [ ]:
memory_manager = MemoryManager(
    api_key=os.environ["OPENAI_API_KEY"],
    provider="openai",
    temperature=0.2,
    business_description="""You are a top-rated personal chef renowned for transforming everyday dishes into gourmet culinary experiences.
    Your specialty lies in elevating familiar comfort foods with sophisticated techniques,
    high-quality ingredients, and artistic presentation.
    Provide creative ideas and detailed instructions
    for turning a classic [dish name] into an elegant, restaurant-quality meal.""",
    include_beliefs=True,
)

print("Tovana MemoryManager initialized.")

## 4. Creating a LangGraph agent with Tovana memory

Let's create our LangGraph agent that incorporates Tovana memory:

In [ ]:
class AgentState(MessagesState):
    user_id: str
    info: Annotated[dict, SharedValue.on("user_id")]

prompt = """You are helpful assistant.

Here is what you know about the user:

<info>
{info}
</info>

Help out the user."""

model = ChatOpenAI()

def call_model(state):
    facts = [d["fact"] for d in state["info"].values()]
    info = "\n".join(facts)
    system_msg = prompt.format(info=info)
    response = model.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    print(f"AI: {response.content}")
    return {
        "messages": [response],
    }

def get_user_input(state: AgentState) -> Dict[str, Any]:
    information_from_stdin = str(input("\nHuman: "))
    return {"messages": [HumanMessage(content=information_from_stdin)]}

def route(state):
    num_human_input = sum(1 for message in state["messages"] if message.type == "human")
    if num_human_input < 5:
        return "not_enough_info"
    if num_human_input == 5:
        return "enough_user_input"
    else:
        return END

def update_memory(state):
    memories = {}
    # Batch update memory with all messages at once
    memory_manager.batch_update_memory(state["user_id"], state["messages"])
    memory_context = memory_manager.get_memory_context(user_id=state["user_id"])
    memories[str(uuid.uuid4())] = {"fact": memory_context}
    print(f"# Tovana memory saved.. \n # {memory_context}")
    return {"messages": [("human", "memory saved")], "info": memories}

memory = MemorySaver()
kv = MemoryStore()

graph = StateGraph(AgentState)
graph.add_node(call_model)
graph.add_node(update_memory)
graph.add_node(get_user_input)

graph.add_edge("update_memory", "call_model")
graph.add_edge(START, "get_user_input")
graph.add_edge("get_user_input", "call_model")
graph.add_conditional_edges(
    "call_model",
    route,
    {
        "not_enough_info": "get_user_input",
        "enough_user_input": "update_memory",
        END: END,
    },
)
graph = graph.compile(checkpointer=memory, store=kv)

print("LangGraph agent with Tovana memory created.")

### Visualizing the LangGraph Agent Flow

The following image represents the flow of our LangGraph agent with Tovana memory:

![LangGraph Tovana Flow](./static/langgraph_cooking_agent.png)

This graph visualization shows the different states and transitions of our agent:

- The process starts at the `_start_` node.
- It then moves to `get_user_input` to collect information from the user.
- The `call_model` node represents where the AI model processes the input and generates a response.
- If there's not enough information, it loops back to `get_user_input` via the `not_enough_info` path.
- When there's enough user input, it proceeds to `update_memory` via the `enough_user_input` path.
- After updating the memory, it returns to `call_model` for further processing.
- The process can end at any point if certain conditions are met, leading to the `_end_` node.

This visual representation helps in understanding the flow of information and decision-making process in our LangGraph agent integrated with Tovana memory. It illustrates how the agent interacts with the user, processes information, updates memory, and makes decisions based on the amount of information gathered.

## 5. Demonstrating cross-thread memory sharing

Now, let's demonstrate how memory can be shared across different threads:

In [ ]:
user_id = str(uuid.uuid4())

print("Starting first thread...")
config = {"configurable": {"thread_id": "1", "user_id": user_id}}
res = graph.invoke({"user_id": user_id}, config=config)

print("\nStarting second thread...")
config = {"configurable": {"thread_id": "2", "user_id": user_id}}
res2 = graph.invoke({"user_id": user_id}, config=config)

print("\nCross-thread memory sharing demonstration complete.")

## 6. Analyzing the results

Let's analyze what happened in our demonstration:

1. We created two separate threads (`thread_id` 1 and 2) for the same user (`user_id`).
2. In each thread, the agent collected information from the user through multiple interactions.
3. After gathering enough information (5 user inputs), Tovana's memory manager updated the user's memory using batch processing.
4. The updated memory was then available in both threads, demonstrating cross-thread memory sharing.

This approach allows for consistent user experiences across different conversation threads, leveraging Tovana's memory capabilities within the LangGraph framework. The batch processing of messages ensures that we capture the full context of the conversation before updating the memory, leading to more accurate and meaningful memory updates.

## Conclusion

In this cookbook, we've explored how to integrate Tovana Memory 🧠 with LangGraph 🦜🕸️, demonstrating:

- Initialization of Tovana MemoryManager
- Creation of a LangGraph agent with Tovana memory
- Cross-thread memory sharing using LangGraph's wider channel memory feature
- Batch processing of messages for efficient and context-aware memory updates

This integration enables more personalized and context-aware AI interactions, allowing your agents to maintain and utilize user-specific information across different conversation threads. The batch processing approach saves processing time and costs while potentially improving the quality of memory updates.

Experiment with this setup to create even more sophisticated AI agents that can learn and adapt from user interactions over time!